Traiter et fusionner des fichiers CSV horaires en regroupant et en complétant les données manquantes par des moyennes et des interpolations, puis enregistrer les résultats sous forme de fichiers CSV pour les directions entrantes et sortantes.

 Ajouter des colonnes de date dérivées et renommer les colonnes pour assurer la cohérence des formats.

In [ ]:
import os
import pandas as pd
import numpy as np

def process_and_merge_files(input_folder, start_date, end_date):
    all_dfs = []      
    
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            print(f"Processing file: {file_path}")
            

            df = pd.read_csv(file_path, parse_dates=['Datetime'])
            
            df = df.groupby(['Datetime', 'Sens'], as_index=False).agg({
                '2R': 'sum', 'VL': 'sum', 'PL': 'sum', 'DayOfWeek': 'first'
            })
            
            df['Hour'] = df['Datetime'].dt.hour
            df['DayOfMonth'] = df['Datetime'].dt.day
            
            full_range = pd.DataFrame(date_range, columns=['Datetime'])
            full_range['DayOfWeek'] = full_range['Datetime'].dt.dayofweek
            full_range['Hour'] = full_range['Datetime'].dt.hour
            full_range['DayOfMonth'] = full_range['Datetime'].dt.day
            full_range['Month'] = full_range['Datetime'].dt.month

            full_range_sens = pd.concat([full_range.assign(Sens=0), full_range.assign(Sens=1)])

            df = pd.merge(full_range_sens, df, on=['Datetime', 'DayOfWeek', 'Hour', 'DayOfMonth', 'Sens'], how='left')

            # Remplir les valeurs manquantes en utilisant la moyenne par DayOfWeek et Hour
            df[['2R', 'VL', 'PL']] = df.groupby(['DayOfWeek', 'Hour','Sens'])[['2R', 'VL', 'PL']].transform(lambda x: x.fillna(x.mean()))
            
            nan_values = df[['2R', 'VL', 'PL']].isna()
            if nan_values.any().any():
                nan_rows = df[nan_values.any(axis=1)]
                print(f"\nNaN values detected in the following rows in file: {filename}")
                print(nan_rows[['Datetime', 'DayOfWeek', 'Hour', 'DayOfMonth', 'Sens', '2R', 'VL', 'PL']])

                # Interpolation linéaire pour combler les NaN
                df[['2R', 'VL', 'PL']] = df[['2R', 'VL', 'PL']].interpolate(method='linear', limit_direction='both')
                nan_values_after_interp = df[['2R', 'VL', 'PL']].isna()

                if nan_values_after_interp.any().any():
                    print(f"\nNaN values remaining after interpolation in file: {filename}")
                    nan_rows_after_interp = df[nan_values_after_interp.any(axis=1)]
                    print(nan_rows_after_interp[['Datetime', 'DayOfWeek', 'Hour', 'DayOfMonth', 'Sens', '2R', 'VL', 'PL']])
                else:
                    print(f"\nNaN values successfully filled by interpolation in file: {filename}")


            # Convertir les colonnes 2R, VL, PL en entiers
            df[['2R', 'VL', 'PL']] = df[['2R', 'VL', 'PL']].round().astype(int)

            post_number = filename.split('.')[0].replace("P", "").zfill(2)  # S'assurer que le numéro de poste a deux chiffres
            
            # Renommer les colonnes avec le bon format
            df.rename(columns={
                '2R': f'2R-P{post_number}', 
                'VL': f'VL-P{post_number}', 
                'PL': f'PL-P{post_number}'}, inplace=True)
            
            all_dfs.append(df)

    
    # Fusionner tous les DataFrames par Datetime, DayOfWeek, Hour, DayOfMonth, et Sens
    merged_df = all_dfs[0]
    for df in all_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on=['Datetime', 'DayOfWeek', 'Hour', 'DayOfMonth','Month', 'Sens'], how='outer')
    
    df_entrante = merged_df[merged_df['Sens'] == 0].drop(columns=['Sens'])
    df_sortante = merged_df[merged_df['Sens'] == 1].drop(columns=['Sens'])
    # Sauvegarder le fichier fusionné final
    df_entrante.to_csv('3month_data_en.csv', index=False)
    df_sortante.to_csv('3month_data_so.csv', index=False)
    print(f"Final merged file saved ")

input_folder = './3types'  
start_date = '2022-10-03' 
end_date = '2023-01-03'  

process_and_merge_files(input_folder, start_date, end_date)


Enrichir un fichier de données de trafic avec des données météorologiques en fusionnant les deux ensembles de données sur la colonne Datetime et en complétant les valeurs manquantes par interpolation. Sauvegarder le fichier enrichi avec un ensemble de colonnes ordonnées dans un fichier CSV de sortie.

In [ ]:
import pandas as pd
from meteostat import Point, Hourly

def enrich_with_meteo(input_file, output_file, location):
    # Lire le fichier d'entrée
    df = pd.read_csv(input_file, parse_dates=['Datetime'])
    
    # Extraire la plage de dates du fichier
    start_date = df['Datetime'].min()
    end_date = df['Datetime'].max()

    # Extraire les données météo pour l'intervalle de dates spécifié
    meteo_data = Hourly(location, start_date, end_date).fetch()

    if meteo_data.empty:
        print("Les données météorologiques sont vides. Assurez-vous que les dates et l'emplacement sont corrects.")
        return

    # Préparer les colonnes de données météorologiques
    meteo_data['DayOfWeek'] = meteo_data.index.dayofweek
    meteo_data['Hour'] = meteo_data.index.hour
    meteo_data['DayOfMonth'] = meteo_data.index.day
    
    meteo_data.rename(columns={
        'temp': 'temperature(degC)',
        'dwpt': 'point_de_rosee(degC)',
        'rhum': 'humidite(%)',
        'prcp': 'precipitations(mm)',
        'snow': 'neige(mm)',
        'wdir': 'vent_direction(deg)',
        'wspd': 'vent_moyen(km/h)',
        'wpgt': 'rafale_vent_max(km/h)',
        'pres': 'pression(hPa)',
        'tsun': 'ensoleillement(h)'  # Correction pour les heures d'ensoleillement
    }, inplace=True)

    meteo_data.reset_index(inplace=True)  # Mettre l'index comme colonne 'Datetime'
    meteo_data.rename(columns={'time': 'Datetime'}, inplace=True)  # Renommer la colonne 'time' en 'Datetime'

    # Fusionner les données de trafic avec les données météorologiques
    df_merged = pd.merge(df, meteo_data, on='Datetime', how='left')
    if 'Hour' not in df_merged.columns:
            df_merged['Hour'] = df_merged['Datetime'].dt.hour
    if 'DayOfWeek' not in df_merged.columns:
            df_merged['DayOfWeek'] = df_merged['Datetime'].dt.dayofweek
    if 'DayOfMonth' not in df_merged.columns:
            df_merged['DayOfMonth'] = df_merged['Datetime'].dt.day
    if 'Month' not in df_merged.columns:
            df_merged['Month'] = df_merged['Datetime'].dt.month
    
    meteo_columns = ['temperature(degC)', 'point_de_rosee(degC)', 'humidite(%)', 'precipitations(mm)', 'neige(mm)', 
                        'vent_direction(deg)', 'vent_moyen(km/h)', 'rafale_vent_max(km/h)', 'pression(hPa)', 'ensoleillement(h)']
        
    df_merged[meteo_columns] = df_merged[meteo_columns].interpolate(method='linear', limit_direction='both')

    ordered_columns = ['Datetime', 'Hour', 'DayOfWeek', 'DayOfMonth', 'Month'] + meteo_columns + [col for col in df_merged.columns if col.startswith('2R') or col.startswith('VL') or col.startswith('PL')]

    df_merged = df_merged[ordered_columns]
    df_merged.to_csv(output_file, index=False)
    
    print(f"Fichier enrichi avec les données météo enregistré avec succès : {output_file}")

input_file = './3month_data_so.csv'  # Le fichier d'entrée avec les données de trafic
output_file = './3month_meteo_data_so.csv'  # Fichier de sortie avec les données météo ajoutées

location = Point(44.8069, -0.6133, 20)

enrich_with_meteo(input_file, output_file, location)


Avant d'effectuer ça, j'ai parcouru manuellement les fichiers pour les mettres sous le meme format, et le meme intervalle.

Par exemple:

In [ ]:
import pandas as pd

df = pd.read_csv('./P25complet.csv')

df['2R'] = df['VELO'] + df['MOTO'] + df['EDPM']
df['PL'] = df['PL_1'] + df['PL_2'] + df['BUS'] + df['UT']
df = df[['Datetime', 'DayOfWeek', 'Sens', '2R', 'VL', 'PL']]
df.to_csv('./3types/P25.csv', index=False)